In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import warnings
pd.options.display.max_columns = None

warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('/kaggle/input/enigma-data/train.csv')

In [ ]:
df


In [ ]:
df.drop(["ID"], axis=1, inplace= True) 

In [ ]:
df.info()

In [ ]:
df['EDUCATION'] = df['EDUCATION'].replace({'graduate school': 1, 'university': 2, 'high school': 3, 'Unknown': 4})

In [ ]:
df['MARITAL_STATUS'] = df['MARITAL_STATUS'].replace({'MARRIED': 1, 'SINGLE': 2, '3': 3})

In [ ]:
df['SEX']=df['SEX'].replace({'MALE': 1, 'FEMALE': 2})

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the categorical features
categorical_cols = ['EDUCATION', 'MARITAL_STATUS', 'SEX']
df_encoded = pd.get_dummies(df, columns=categorical_cols)

X_encoded = df_encoded.drop(['DEFAULT.PAYMENT.NEXT.MONTH'], axis=1)  # Independent features
y_encoded = df_encoded['DEFAULT.PAYMENT.NEXT.MONTH']  # Dependent feature

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_encoded_scaled = scaler.fit_transform(X_encoded) 

from sklearn.model_selection import train_test_split
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded_scaled, y_encoded, test_size=0.20, random_state=42)

# Checking the class distribution before oversampling
from collections import Counter
print("Before oversampling: ", Counter(y_train_encoded))  # Display class distribution before oversampling

# Applying SMOTE (Synthetic Minority Over-sampling Technique) for oversampling
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_resampled_encoded, y_train_resampled_encoded = smote.fit_resample(X_train_encoded, y_train_encoded)  # Apply SMOTE

# Checking the class distribution after oversampling
print("After oversampling: ", Counter(y_train_resampled_encoded))  # Display class distribution after oversampling

In [ ]:
# Use the resampled data for training
X_train = X_train_resampled_encoded
y_train = y_train_resampled_encoded

# Use the original testing data for evaluation
X_test = X_test_encoded
y_test = y_test_encoded

In [ ]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

In [ ]:
test=pd.read_csv("/kaggle/input/testtest/test (1).csv")

In [ ]:
test['EDUCATION'] = test['EDUCATION'].replace({'graduate school': 1, 'university': 2, 'high school': 3, 'Unknown': 4})

In [ ]:
test['MARITAL_STATUS'] = test['MARITAL_STATUS'].replace({'MARRIED': 1, 'SINGLE': 2, '3': 3})

In [ ]:
test['SEX']=test['SEX'].replace({'MALE': 1, 'FEMALE': 2})

In [ ]:
test=test.drop(columns=["ID"])

In [ ]:
test = pd.get_dummies(test, columns=['SEX', 'MARITAL_STATUS', 'EDUCATION'])

test = test[X_encoded.columns]


In [ ]:
test = scaler.transform(test)

In [ ]:
pip install catboost

In [ ]:
import optuna

def optimize_hp(trial):
    cb_params = {
        'iterations': 50,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.1, 0.2),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.1, 20.0),
        'random_strength': trial.suggest_float('random_strength', 1.0, 2.0),
        'depth': trial.suggest_int('depth', 6, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        "use_best_model": True,
        "task_type": "GPU",
        'random_seed': 42
    }
    
    model = CatBoostClassifier(**cb_params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
study = optuna.create_study(direction='maximize')
study.optimize(optimize_hp, n_trials=500)

# Get the best hyperparameters
best_params = study.best_params
best_f1_score = study.best_value

In [ ]:
best_params

In [ ]:
best_f1_score

In [ ]:
from catboost import CatBoostClassifier

# Use the parameters motivated by the ones generated by Octuna
best_params={'iterations': 1000, 'learning_rate': 0.10004428063100487, 'depth': 6, 'l2_leaf_reg': 0.9914, 'bagging_temperature': 1.4156789415350013, 'random_strength': 4.206634177846974, 'min_data_in_leaf': 81, 'eval_metric': 'F1', 'random_seed': 42, 'verbose': 100}
# Define the CatBoost classifier with the best parameters
catboost_model = CatBoostClassifier(**best_params)

# Train the model
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Make predictions on the test set
pred_catboost = catboost_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, pred_catboost)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, pred_catboost))

In [ ]:
# Calculate accuracy
f1 = f1_score(y_test, pred_catboost)
print("Accuracy:", f1)

In [ ]:
DATA1=pd.DataFrame(preduc1)

In [ ]:
DATA1.to_csv('predictions_pending.csv')

In [ ]:
catboost_model.save_model('catboost_model1.cbm')